<a href="https://colab.research.google.com/github/chinmay002/LLM-s/blob/main/Llama_index_with_Pinecode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU llama-index datasets pinecone-client openai transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 854.1/854.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.9/217.9 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [8]:
import pandas as pd
import numpy as np
from datasets import load_dataset
data = load_dataset('squad',split = 'train')
data = data.to_pandas()[['id','context','title']]
data.drop_duplicates(subset = 'context',keep = 'first',inplace=True)
data.head()

,id,context,title
0,5733be284776f41900661182,"Architecturally, the school has a Catholic cha...",University_of_Notre_Dame
5,5733bf84d058e614000b61be,"As at most other universities, Notre Dame's st...",University_of_Notre_Dame
10,5733bed24776f41900661188,The university is the major seat of the Congre...,University_of_Notre_Dame
15,5733a6424776f41900660f51,The College of Engineering was established in ...,University_of_Notre_Dame
20,5733a70c4776f41900660f64,All of Notre Dame's undergraduate students are...,University_of_Notre_Dame


In [9]:
#transform into document object

from llama_index import Document
docs = []
for i , row in data.iterrows():
  docs.append(Document(text = row['context'],
                       doc_id = row['id'],
                       extra_info = {'title':row['title']}))

In [11]:
docs[0]

Document(id_='5733be284776f41900661182', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', start_char_idx=None, end_char_idx=N

In [14]:
len(docs)

18891

In [ ]:
#SimpleNodeParser process our list of Doc obj into nodes which are basic units that llama_index uses for indexing and querying .


In [18]:
import os
os.environ['OPENAI_API_KEY'] = '

from llama_index.node_parser import SimpleNodeParser
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
nodes[0]

TextNode(id_='d2a7b091-d0e7-4d49-bffc-65f495c8c722', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5733be284776f41900661182', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'title': 'University_of_Notre_Dame'}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6')}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where th

In [21]:
docs[0]

Document(id_='5733be284776f41900661182', embedding=None, metadata={'title': 'University_of_Notre_Dame'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='ba8eb7d843763d4f7e7f71f0be64c70c4597fd712bfcce77007fc4c821b8d1b6', text='Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.', start_char_idx=None, end_char_idx=N

#INDEXING
#Pinecode

In [23]:
import pinecone
os.environ['PINECONE_API_KEY'] = '
os.environ['PINECONE_ENVIRONMENR'] = 'gcp-starter'

In [32]:
pinecone.list_indexes()


[]

In [33]:
#initialize pincone

pinecone.init(api_key = os.environ['PINECONE_API_KEY'] ,environment = os.environ['PINECONE_ENVIRONMENR'])

#create the index if it does not exist already
index_name = 'llama-index-intro'
if index_name not in pinecone.list_indexes():
  pinecone.create_index(name=index_name,metric= 'cosine',dimension = 1536)

pinecone_index = pinecone.Index(index_name)


In [36]:
from llama_index.vector_stores import PineconeVectorStore
#initialize the namespalce  ;Think of it as name given to index which conatins some data, like grouping similar data into one index and giving name namespalce
namespace = ''#deafualt
vectore_store = PineconeVectorStore(pinecone_index = pinecone_index)
#this will serve as the storage and retreival interface for our doc embed in pine vectore database


In [37]:
#We use GPTVectorstoreIndex with docs,using pincevectorstoreand openAIEmbeddnigs
#GPTVectorstoreindex handles the indexing and querying process

from llama_index import GPTVectorStoreIndex,StorageContext,ServiceContext
from llama_index.embeddings.openai import OpenAIEmbedding


#setupd teh storage
storage_context =  StorageContext.from_defaults(vector_store = vectore_store)

embed_model = OpenAIEmbedding(model = 'text-embedding-ada-002',embed_batch_size=1000)
#setup the embed model
service_context = ServiceContext.from_defaults(embed_model= embed_model)
index = GPTVectorStoreIndex.from_documents(documents = docs,storage_context = storage_context,service_context = service_context)

Upserted vectors:   0%|          | 0/18891 [00:00<?, ?it/s]

In [38]:
query_engine = index.as_query_engine()
res = query_engine.query("what year college of engineering established in Notre Dame")
print(res)

The College of Engineering was established in 1920.
